In [ ]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive/')

# 目的の場所（フォルダ・ディレクトリ）へ移動（各自の環境で適宜修正）
%cd "/content/drive/MyDrive/Colab Notebooks/JKJ1A/"
%ls

# 深層学習モデルの訓練の概観

ここではPyTorchを用いて深層学習モデルの訓練を行うコードの様子を見ます．

スライドで説明した機械学習の3要素は以下の通りでした．

- **データセット**: モデルを訓練するためのデータセット．ここではCIFAR-10データセットを用います．
- **学習モデル**: ここでは深層学習モデルのことです．特にVGG11モデルを用います．
- **最適化**: いわゆる学習（モデルの更新）のことです．ここでは確率的勾配法(SGD)を用います．

以下のコードで，どのコードがどれに対応するかを追って行きます．

In [6]:
# 三要素の１：データの準備
from src.cifar10 import load_data                                    # src/cifar10.pyを利用
trainloader, testloader, classes = load_data(256, use_all=True)  # 全てのデータをつかう場合は use_all=True

Files already downloaded and verified
Files already downloaded and verified


In [9]:
# 三要素の２：モデルの準備
from src.models import VGG             # src/vgg.pyを利用
model = VGG('VGG11')            # ネットワークを取得
model.cuda()                    # GPUへ移動
# model.to('mps')                   # GPU付きのMacでやる場合はこっち
''

''

In [12]:
# 三要素の３：学習
from time import time 

# 1エポック分の操作．学習データを1周する
def train_step(model, dataloader, optimizer, criterion):
    mean_loss = 0
    for x, y in dataloader:                # dataloader からデータをひと束取り出す．
        x, y = x.cuda(), y.cuda()          # データをGPUに移す
        # x, y = x.to('mps'), y.to('mps')    # データをGPUに移す (GPU付きのMac)
        
        optimizer.zero_grad()              # 勾配の初期化

        output = model(x)                  # ネットワークにデータxを入れて，出力outputを得る 
        loss = criterion(output, y)        # 誤差の計算（出力output と 正解y の比較）

        loss.backward()                    # 勾配の計算
        optimizer.step()                   # 勾配法（SGD）を行う
        
        mean_loss += loss.item()           # item()とすると要素1のテンソルを数値に変換できる．
    
    mean_loss /= len(dataloader)           # 全データでの平均誤差
    return mean_loss


# 上の1エポック分の操作を繰り返す
def train(model, dataloader, optimizer, criterion, n_epochs):
    print('| Epoch    | Loss    | Elapsed time [mins]')
    
    start_time = time()
    for epoch in range(n_epochs):
        print(f'  {epoch:03}', end='')
        
        loss = train_step(model, trainloader, optimizer, criterion)
        runtime = time() - start_time
        
        print(f'\t     {loss:.3f}     {runtime/60:.2f}')

In [13]:
%%time
# 学習の実行
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()                         # 誤差関数をセット
optimizer = optim.SGD(model.parameters(), lr=0.01)          # 確率的勾配法をセット（第一引数にモデルのパラメタを与えてるのに注意）

nepochs = 2                                               # 学習データを何周するか
train(model, trainloader, optimizer, criterion, nepochs)  # 学習を実行

| Epoch    | Loss    | Elapsed time [mins]
  000	     1.295     0.46
  001	     0.875     0.90
CPU times: user 14.2 s, sys: 3.6 s, total: 17.8 s
Wall time: 53.7 s


### 備考
まとめると，以下のようになります．
- データセットを変えたいなら，`load_data`の中身を変える
- モデルを変えるなら，`Net`の中身を変える（今回は`VGG`）
- 訓練方法を変えるなら，`train`（特に`train_step`）を変える

Dat1-2-CIFAR-10では，それぞれの中身をもう少し詳細に見ていきます．